In [1]:
import os,sys
import shutil,glob
import pickle
import path
from tqdm import tqdm
import hashlib

In [2]:
rearrange_dir = '../RAID5/'

In [51]:
all_files = glob.glob(rearrange_dir+'**/*.tar.gz', recursive=True)
all_files

['../RAID5/honeypot/2020_01_25/cowrie/台固媒體/cowrie_log_2020_01_25.tar.gz',
 '../RAID5/honeypot/2020_01_25/cowrie/台固媒體/cowrie_file_2020_01_25.tar.gz',
 '../RAID5/honeypot/2020_01_25/cowrie/中嘉寬頻/cowrie_log_2020_01_25.tar.gz',
 '../RAID5/honeypot/2020_01_25/cowrie/中嘉寬頻/cowrie_file_2020_01_25.tar.gz',
 '../RAID5/honeypot/2020_01_25/cowrie/中華電信/cowrie_log_2020_01_25.tar.gz',
 '../RAID5/honeypot/2020_01_25/cowrie/中華電信/cowrie_file_2020_01_25.tar.gz',
 '../RAID5/honeypot/2020_01_25/glastopf/台固媒體/glastopf_log_2020_01_25.tar.gz',
 '../RAID5/honeypot/2020_01_25/glastopf/台固媒體/glastopf_file_2020_01_25.tar.gz',
 '../RAID5/honeypot/2020_01_25/glastopf/中嘉寬頻/glastopf_log_2020_01_25.tar.gz',
 '../RAID5/honeypot/2020_01_25/glastopf/中嘉寬頻/glastopf_file_2020_01_25.tar.gz',
 '../RAID5/honeypot/2020_01_25/glastopf/中華電信/glastopf_log_2020_01_25.tar.gz',
 '../RAID5/honeypot/2020_01_25/glastopf/中華電信/glastopf_file_2020_01_25.tar.gz',
 '../RAID5/honeypot/2020_01_25/dionaea/台固媒體/dionaea_log_2020_01_25.tar.gz',
 '../R

.split('/') :
* 4=>pcap/hp
* 3=>ISP (轉換)
* 5=>date (注意格式)
* 6=>fileName
***
#### 最原本.tar.gz/.json TTC所給之原始檔案目前位於RAID5目錄下
* pcap 
    * date
        * ISP
            * fileName
* honeypot 
    * date
        * type
            * ISP or mongoDB
                * fileName

#### 經解壓縮、加工處理後等大size個別資料目前位於 data_hdd4t目錄下
* pcap_process
    * date
        * ISP
            * fileName
* honeypot_process
    * date
        * type
            * ISP or mongoDB
                * fileName

#### spark dataframe、pickle等經過整合或小size資料目前位於spark_data目錄下
* pcap_inter
    * date
        * ISP
            * fileName
* honeypot_inter
    * date
        * type
            * ISP or mongoDB
                * fileName

In [3]:
def aliase_convert(s):
    '''
    dictionary baesd conversion
    '''
    if s == '台哥大':
        return '台灣大哥大'
    elif s == '台基開發':
        return '台灣基礎開發'
    elif s == '亞太':
        return '亞太電信'
    elif s == '遠傳':
        return '遠傳電信'
    else:
        return s

In [53]:
for path in tqdm(all_files):
    path_li = path.split('/')
    if path_li[2] in ['pcap','honeypot','metadata']:
        continue
    if len(path_li) == 7: #pcap
        date_dir = path_li[-2]
        date_dir = date_dir.replace('-','_')
        assert len(date_dir.split('_')) == 3
        isp_dir = path_li[3]
        isp_dir = aliase_convert(isp_dir)
        fileName = path_li[-1]
        assert 'pcap' in fileName
        assert date_dir.split('_')[-1] in fileName
        assert '.tar.gz' in fileName
        save_dir = rearrange_dir+'pcap/'+date_dir+'/'+isp_dir#+'/'+fileName
        if not os.path.exists(save_dir):
            os.makedirs(save_dir,exist_ok=True)
        try:
            shutil.move(path,save_dir+'/')
        except :
            print(path)
            ori = hashlib.md5(open(path,'rb').read()).hexdigest()
            dest =  hashlib.md5(open(rearrange_dir+'pcap/'+date_dir+'/'+isp_dir+'/'+fileName,'rb').read()).hexdigest()
            if ori == dest:
                os.remove(path)
            else:
                print('_2 was created')                
                shutil.move(path,save_dir+'/'+fileName+'_2')
    elif len(path_li) == 8: #hp
        date_dir = path_li[-2]
        date_dir = date_dir.replace('-','_')
        assert len(date_dir.split('_')) == 3
        type_dir = path_li[-3]
        assert type_dir in ['amun', 'cowrie', 'dionaea',  'glastopf']
        isp_dir = path_li[3]
        isp_dir = aliase_convert(isp_dir)
        fileName = path_li[-1]
        assert type_dir in fileName
        assert date_dir.split('_')[-1] in fileName
        assert '.tar.gz' in fileName
        save_dir = rearrange_dir+'honeypot/'+date_dir+'/'+type_dir+'/'+isp_dir#+'/'+fileName
        if not os.path.exists(save_dir):
            os.makedirs(save_dir,exist_ok=True)        
        try:
            shutil.move(path,save_dir+'/')
        except : #重複一模一樣的檔名
            print(path)
            ori = hashlib.md5(open(path,'rb').read()).hexdigest()
            dest =  hashlib.md5(open(save_dir+'/'+fileName,'rb').read()).hexdigest()
            if ori == dest: #真的一模一樣舊移除舊的
                os.remove(path)
            else: #不一樣舊都保留到新的位置去
                print('_2 was created')
                shutil.move(path,save_dir+'/'+fileName+'_2')
            


 70%|███████   | 2160/3079 [04:04<01:18, 11.69it/s]

../RAID5/0112_0125/台灣大哥大/pcap/2020_01_12/pcap_2020_01_12.tar.gz



 86%|████████▌ | 2645/3079 [12:59<53:59,  7.46s/it]  

../RAID5/0112_0125/台灣碩網/pcap/2020_01_12/pcap_2020_01_12.tar.gz



 89%|████████▉ | 2745/3079 [15:04<14:41,  2.64s/it]

../RAID5/0112_0125/亞太電信/pcap/2020_01_12/pcap_2020_01_12.tar.gz



100%|██████████| 3079/3079 [15:07<00:00,  3.39it/s] 


In [4]:
def recursive_delete_if_empty(path):
    """Recursively delete empty directories; return True
    if everything was deleted."""

    if not os.path.isdir(path):
        # If you also want to delete some files like desktop.ini, check
        # for that here, and return True if you delete them.
        return False

    # Note that the list comprehension here is necessary, a
    # generator expression would shortcut and we don't want that!
    if all([recursive_delete_if_empty(os.path.join(path, filename))
            for filename in os.listdir(path)]):
        # Either there was nothing here or it was all deleted
        os.rmdir(path)
        return True
    else:
        return False

In [56]:
one = recursive_delete_if_empty(rearrange_dir+'0112_0125')
two = recursive_delete_if_empty(rearrange_dir+'0126_0208')
three = recursive_delete_if_empty(rearrange_dir+'1229_0112')
print(one,two,three)

False False False


### Mongodb log from  Mybook

In [14]:
src_dir = '../RAID5/Mongodblog/'

In [15]:
all_paths = glob.glob(src_dir+'**/*.json', recursive=True)
print(len(all_paths))
all_paths

757


['../RAID5/Mongodblog/遠傳電信/amun/amun.events_2019-12-29.json',
 '../RAID5/Mongodblog/台灣大哥大/glastopf/glastopf.files_2020-01-11.json',
 '../RAID5/Mongodblog/台灣大哥大/glastopf/glastopf.files_2020-01-10.json',
 '../RAID5/Mongodblog/台灣大哥大/glastopf/glastopf.files_2020-01-09.json',
 '../RAID5/Mongodblog/台灣大哥大/glastopf/glastopf.files_2020-01-08.json',
 '../RAID5/Mongodblog/台灣大哥大/glastopf/glastopf.files_2020-01-07.json',
 '../RAID5/Mongodblog/台灣大哥大/glastopf/glastopf.files_2020-01-06.json',
 '../RAID5/Mongodblog/台灣大哥大/glastopf/glastopf.files_2020-01-05.json',
 '../RAID5/Mongodblog/台灣大哥大/glastopf/glastopf.files_2020-01-04.json',
 '../RAID5/Mongodblog/台灣大哥大/glastopf/glastopf.files_2020-01-03.json',
 '../RAID5/Mongodblog/台灣大哥大/glastopf/glastopf.files_2020-01-02.json',
 '../RAID5/Mongodblog/台灣大哥大/glastopf/glastopf.files_2020-01-01.json',
 '../RAID5/Mongodblog/台灣大哥大/glastopf/glastopf.files_2019-12-31.json',
 '../RAID5/Mongodblog/台灣大哥大/glastopf/glastopf.files_2019-12-30.json',
 '../RAID5/Mongodblog/台灣大哥大/

In [16]:
for path in tqdm(all_paths):
    path_li = path.split('/')
    #以下要隨路徑而改
    date_dir = path_li[-1].split('_')[-1].split('.')[0]
    date_dir = date_dir.replace('-','_')
    assert len(date_dir.split('_')) == 3
    type_dir = path_li[-2]
    assert type_dir in ['amun', 'cowrie', 'dionaea',  'glastopf']
    isp_dir = path_li[-3]
    isp_dir = aliase_convert(isp_dir)
    fileName = path_li[-1]
    assert type_dir in fileName
    assert date_dir.split('_')[-1] in fileName
    assert '.json' in fileName
    save_dir = rearrange_dir+'honeypot/'+date_dir+'/'+type_dir+'/mongoDB/'+isp_dir#+'/'+fileName
    if not os.path.exists(save_dir):
        os.makedirs(save_dir,exist_ok=True)        
    try:
        shutil.move(path,save_dir+'/')
    except : #重複一模一樣的檔名
        print(path)
        ori = hashlib.md5(open(path,'rb').read()).hexdigest()
        dest =  hashlib.md5(open(save_dir+'/'+fileName,'rb').read()).hexdigest()
        if ori == dest: #真的一模一樣舊移除舊的
            pass
            os.remove(path)
        else: #不一樣舊都保留到新的位置去
            print('_2 was created')
            shutil.move(path,save_dir+'/'+fileName+'_2')    

  0%|          | 1/757 [00:00<01:19,  9.49it/s]

../RAID5/Mongodblog/遠傳電信/amun/amun.events_2019-12-29.json


100%|██████████| 757/757 [00:04<00:00, 169.03it/s]


In [17]:
recursive_delete_if_empty(src_dir)

False

## original old data

In [29]:
src_dir = '../data_hdd4t/中華電信/'

In [20]:
all_files = glob.glob(src_dir+'**/*.tar.gz', recursive=True)
print(len(all_files))
all_files

635


['../data_hdd4t/亞太/honeypot/2020-01-10/cowrie/cowrie_log_2020-01-10.tar.gz',
 '../data_hdd4t/亞太/honeypot/2020-01-10/cowrie/cowrie_file_2020-01-10.tar.gz',
 '../data_hdd4t/亞太/honeypot/2020-01-10/amun/amun_log_2020-01-10.tar.gz',
 '../data_hdd4t/亞太/honeypot/2020-01-10/amun/amun_file_2020-01-10.tar.gz',
 '../data_hdd4t/亞太/honeypot/2020-01-10/glastopf/glastopf_file_2020-01-10.tar.gz',
 '../data_hdd4t/亞太/honeypot/2020-01-10/glastopf/glastopf_log_2020-01-10.tar.gz',
 '../data_hdd4t/亞太/honeypot/2020-01-10/dionaea/dionaea_log_2020-01-10.tar.gz',
 '../data_hdd4t/亞太/honeypot/2020-01-10/dionaea/dionaea_file_2020-01-10.tar.gz',
 '../data_hdd4t/亞太/honeypot/2020-01-11/cowrie/cowrie_log_2020-01-11.tar.gz',
 '../data_hdd4t/亞太/honeypot/2020-01-11/cowrie/cowrie_file_2020-01-11.tar.gz',
 '../data_hdd4t/亞太/honeypot/2020-01-11/amun/amun_log_2020-01-11.tar.gz',
 '../data_hdd4t/亞太/honeypot/2020-01-11/amun/amun_file_2020-01-11.tar.gz',
 '../data_hdd4t/亞太/honeypot/2020-01-11/glastopf/glastopf_file_2020-01-11.t

In [24]:
for path in tqdm(all_files):
    path_li = path.split('/')
    if path_li[2] in ['pcap','honeypot','metadata']:
        continue
    if len(path_li) == 5: #pcap
        date_dir = path_li[-1].split('.')[1]
        date_dir = date_dir.replace('-','_')
        assert len(date_dir.split('_')) == 3
        isp_dir = path_li[2]
        isp_dir = aliase_convert(isp_dir)
        fileName = path_li[-1]
        assert 'snort' in fileName
        assert date_dir.split('_')[-1] in fileName
        assert '.tar.gz' in fileName
        save_dir = rearrange_dir+'pcap/'+date_dir+'/'+isp_dir#+'/'+fileName
        if not os.path.exists(save_dir):
            os.makedirs(save_dir,exist_ok=True)
        try:
            shutil.move(path,save_dir+'/')
        except :
            print(path)
            ori = hashlib.md5(open(path,'rb').read()).hexdigest()
            dest =  hashlib.md5(open(rearrange_dir+'pcap/'+date_dir+'/'+isp_dir+'/'+fileName,'rb').read()).hexdigest()
            if ori == dest:
                os.remove(path)
            else:
                print('_2 was created')                
                shutil.move(path,save_dir+'/'+fileName+'_2')
    elif len(path_li) == 7: #hp
        date_dir = path_li[-3]
        date_dir = date_dir.replace('-','_')
        assert len(date_dir.split('_')) == 3
        type_dir = path_li[-2]
        assert type_dir in ['amun', 'cowrie', 'dionaea',  'glastopf']
        isp_dir = path_li[2]
        isp_dir = aliase_convert(isp_dir)
        fileName = path_li[-1]
        assert type_dir in fileName
        assert date_dir.split('_')[-1] in fileName
        assert '.tar.gz' in fileName
        save_dir = rearrange_dir+'honeypot/'+date_dir+'/'+type_dir+'/'+isp_dir#+'/'+fileName
        if not os.path.exists(save_dir):
            os.makedirs(save_dir,exist_ok=True)        
        try:
            shutil.move(path,save_dir+'/')
        except : #重複一模一樣的檔名
            print(path)
            ori = hashlib.md5(open(path,'rb').read()).hexdigest()
            dest =  hashlib.md5(open(save_dir+'/'+fileName,'rb').read()).hexdigest()
            if ori == dest: #真的一模一樣舊移除舊的
                os.remove(path)
            else: #不一樣舊都保留到新的位置去
                print('_2 was created')
                shutil.move(path,save_dir+'/'+fileName+'_2')
            

 17%|█▋        | 106/635 [09:35<02:22,  3.72it/s]  

../data_hdd4t/台灣碩網/honeypot/2020-01-03/cowrie/cowrie_log_2020-01-03.tar.gz
../data_hdd4t/台灣碩網/honeypot/2020-01-03/amun/amun_file_2020-01-03.tar.gz


 17%|█▋        | 107/635 [09:36<02:09,  4.09it/s]

../data_hdd4t/台灣碩網/honeypot/2020-01-03/amun/amun_log_2020-01-03.tar.gz


 17%|█▋        | 108/635 [09:37<03:52,  2.27it/s]

...

 75%|███████▌  | 477/635 [4:01:22<04:52,  1.85s/it]

../data_hdd4t/中華電信/honeypot/2020-01-01/amun/amun_log_2020-01-01.tar.gz
../data_hdd4t/中華電信/honeypot/2020-01-01/glastopf/glastopf_file_2020-01-01.tar.gz
../data_hdd4t/中華電信/honeypot/2020-01-01/glastopf/glastopf_log_2020-01-01.tar.gz
../data_hdd4t/中華電信/honeypot/2020-01-01/dionaea/dionaea_log_2020-01-01.tar.gz


 76%|███████▌  | 481/635 [4:01:23<03:23,  1.32s/it]

../data_hdd4t/中華電信/honeypot/2020-01-01/dionaea/dionaea_file_2020-01-01.tar.gz


100%|██████████| 635/635 [7:07:56<00:00, 40.43s/it]    


In [28]:
recursive_delete_if_empty(src_dir)

False

In [27]:
src_dir

'../data_hdd4t/'

In [23]:
len('../data_hdd4t/台灣碩網/pcap/snort.2020-01-06.tar.gz'.split('/'))

5